<a href="https://colab.research.google.com/github/hane-momiji/IndustrialOrganization/blob/master/Problemset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.RandomState(1234)

# Chapter 1: Import data


### Data structure
1. Firm ID
2. Periods
3. State
4. Action

Thus, we'll calculate the transition probability of each transition and make new variable. To do so, we add
5. Next_state
7. Log-transition probability
6. Transition probability

### Transition Probability

$$
P_0 = 
\begin{align}
\begin{bmatrix}
1 & 0 & 0 & 0 & 0\\
\kappa & 1 - \kappa & 0 & 0 & 0\\
0 & \kappa &  1- \kappa & 0 & 0\\
0 & 0 & \kappa & 1- \kappa & 0\\
0 & 0 & 0 & \kappa & 1- \kappa 
\end{bmatrix} 
\end{align}
$$

$$
\begin{align}
P_1 = 
\begin{bmatrix}
1-\gamma & \gamma & 0 & 0 & 0\\
\kappa & 1 - \kappa - \gamma & \gamma & 0 & 0\\
0 & \kappa & 1- \kappa - \gamma & \gamma & 0\\
0 & 0 & \kappa & 1 - \kappa - \gamma & \gamma\\
0 & 0 & 0 & \kappa &  1 - \kappa 
\end{bmatrix}
\end{align}
$$

### Term profit of firm
$$
\pi(s_t, a_t) = \alpha \ln(s_t) - \beta a_t + \varepsilon(a_t)
$$

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dat = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_IO/PS2019GIO2data1.csv', names = ["firmID", "time", "state", "action"])
dat.tail(100)

,firmID,time,state,action
99900,1000,1,5,0
99901,1000,2,5,0
99902,1000,3,5,0
99903,1000,4,5,0
99904,1000,5,5,1
99905,1000,6,5,0
99906,1000,7,5,0
99907,1000,8,5,0
99908,1000,9,5,0
99909,1000,10,5,0


### Define parameters  and give init values


In [0]:
delta = 0.95
kappa = 0.2
gamma = 0.1
theta3 = [kappa, gamma]

# Transition prob. matrices
P0 = np.eye(5, dtype = float) - theta3[0]*np.eye(5, dtype = float)
P0[0,0] = 1
kap = np.eye(4, dtype = float)*theta3[0]
kap = np.insert(np.insert(kap, obj = 0, values = 0, axis = 0), obj = 4, values = 0, axis = 1) # 4 * 4 -> 5 * 4 -> 5 * 5
P0 += kap # defined transition prob. matrix when a = 0

gam = np.eye(4, dtype = float)*theta3[1]
gam_center = np.insert(np.insert(gam, obj = 4, values = 0, axis = 0), obj = 4, values = 0, axis = 1)
gam_up = np.insert(np.insert(gam, obj = 0, values = 0, axis = 1), obj = 4, values = 0, axis = 0)
P1 = P0 - gam_center + gam_up # defined transition prob. matrix when a = 1
print(P0)
print(P1) 
P2 = np.array(range(25))
P2 = P2.reshape(5,5)
print(P2)

[[1.  0.  0.  0.  0. ]
 [0.2 0.8 0.  0.  0. ]
 [0.  0.2 0.8 0.  0. ]
 [0.  0.  0.2 0.8 0. ]
 [0.  0.  0.  0.2 0.8]]
[[0.9 0.1 0.  0.  0. ]
 [0.2 0.7 0.1 0.  0. ]
 [0.  0.2 0.7 0.1 0. ]
 [0.  0.  0.2 0.7 0.1]
 [0.  0.  0.  0.2 0.8]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


#### data processing for describing the transition

In [0]:
def np_log(x):
    return np.log(np.clip(x, a_min = 1e-8, a_max = x)) 

dat['next_state'] = dat['state'].shift(periods = -1, axis = 0)
data_trans = dat[dat['time'] != 100]
data_trans = data_trans.astype('int8')
# data_trans['logL'] = data_trans
# data_trans = data_trans.values
data_trans['logL'] = np.where(data_trans['action'] == 1, np_log(P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), np_log(P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
data_trans['L'] = np.where(data_trans['action'] == 1, (P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), (P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
data_trans['label'] = np.where(data_trans['action'] == 1, (P2[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), (P2[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
# data_trans.loc[data_trans['action'] == 1, 'logL'] = np_log(P1[(data_trans['state'] - 1), (data_trans['next_state']-1) ])
# data_trans.loc[data_trans['action'] == 0, 'logL'] = np_log(P0[ (data_trans['state'] -1), (data_trans['next_state']-1)])
# data_trans.loc[data_trans['action'] == 0, 'L'] = P2[(data_trans['state'] - 1), (data_trans['next_state'] - 1 )]
# data_trans.loc[data_trans['action'] == 1, 'L'] = P2[(data_trans['state'] - 1), (data_trans['next_state']-1) ]
# data_trans['L'] =  P2[(data_trans['state'] - 1), (data_trans['next_state']-1) ]
# for i in range(len(data_trans)):
#     if (data_trans[i, 3] == 0):
#         data_trans['logL'][i] = P0[(data_trans[i, 2]), (data_trans[i,4])]
#     else:
#         data_trans['logL'][i] = P1[(data_trans[i, 2]), (data_trans[i, 4])]
logL = data_trans['logL'].sum()

In [0]:
data_trans.head(10)

,firmID,time,state,action,next_state,logL,L,label
0,1,1,2,1,3,-2.302585,0.1,7
1,1,2,3,0,3,-0.223144,0.8,12
2,1,3,3,0,3,-0.223144,0.8,12
3,1,4,3,0,3,-0.223144,0.8,12
4,1,5,3,0,3,-0.223144,0.8,12
5,1,6,3,0,3,-0.223144,0.8,12
6,1,7,3,0,2,-1.609438,0.2,11
7,1,8,2,0,2,-0.223144,0.8,6
8,1,9,2,1,3,-2.302585,0.1,7
9,1,10,3,1,4,-2.302585,0.1,13


# Chapter 2: Estimate the transition probability

In [0]:
def NLL(theta3):
    # Transition prob. matrices
    P0 = np.eye(5, dtype = float) - theta3[0]*np.eye(5, dtype = float)
    P0[0,0] = 1
    kap = np.eye(4, dtype = float)*theta3[0]
    kap = np.insert(np.insert(kap, obj = 0, values = 0, axis = 0), obj = 4, values = 0, axis = 1) # 4 * 4 -> 5 * 4 -> 5 * 5
    P0 += kap # defined transition prob. matrix when a = 0

    gam = np.eye(4, dtype = float)*theta3[1]
    gam_center = np.insert(np.insert(gam, obj = 4, values = 0, axis = 0), obj = 4, values = 0, axis = 1)
    gam_up = np.insert(np.insert(gam, obj = 0, values = 0, axis = 1), obj = 4, values = 0, axis = 0)
    P1 = P0 - gam_center + gam_up # defined transition prob. matrix when a = 1
    dat['next_state'] = dat['state'].shift(periods = -1, axis = 0)
    data_trans = dat[dat['time'] != 100]
    data_trans = data_trans.astype('int8')
    data_trans['logL'] = np.where(data_trans['action'] == 1, np_log(P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), np_log(P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
    data_trans['L'] = np.where(data_trans['action'] == 1, (P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), (P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
    data_trans['label'] = np.where(data_trans['action'] == 1, (P2[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), (P2[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
    logL = data_trans['logL'].sum()
    return -logL


### Optimization
Pythonでの最適化はScipyのoptimizeパッケージを用いる。詳しい解説は[こちら](http://www.turbare.net/transl/scipy-lecture-notes/intro/scipy.html)

多変数関数の最適化については、[公式の説明](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html)が詳しい。

#### Linear constraints:

\begin{align}
\begin{bmatrix}
0 \\
-1 \\
-1
\end{bmatrix}
\leq
\begin{bmatrix}
1 & 1 \\
-1 & 0 \\
0 & -1 
\end{bmatrix}
\begin{bmatrix}
\kappa \\
\gamma
\end{bmatrix}
\leq
\begin{bmatrix}
1 \\
0 \\
0 
\end{bmatrix}
\end{align}

In [0]:
from scipy.optimize import minimize
# from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint

lin_const = LinearConstraint([[1, 1], [-1, 0], [0, -1]], [0, -1, -1], [1, 0, 0])
result = minimize(NLL, [0.1, 0.2], method = 'trust-constr', constraints = lin_const) # これはなぜ要素が２つなのか...?

/usr/local/lib/python3.6/dist-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


In [0]:
print(result.x)


[0.09103848 0.53004492]


### Estimation result
$$
\hat{\kappa} = 0.09103848, \qquad \hat{\gamma} = 0.53004492
$$

# Chapter 3: Let's compute Expected Value Function!


## Estimation procedure
We want to estimate $\theta_1$, that is maintenance cost parameters.
1. Given $\theta_1 = \bar{\theta}$, compute the expected value function by finding fixed point of $EV(s, a)$
2. Given the value function, evaluate the objective function, which is given by
$$
\begin{gather*}
\sum_{i = 1}^{N} \sum_{t = 1}^{T} 
\log\left[
P(a_{it} = 1 \mid s_t, \theta)^{a_{it}} \cdot P(a_{it} = 0 \mid s_t, \theta)^{1 - a_{it}}
\right]\\
\sum_{i = 1}^{N} \sum_{t = 1}^{T} \left\{
a_{it} \log\left[
P(a_{it} = 1 \mid s_t, \theta)\right] 
+ (1- a_{it})\log \left[P(a_{it} = 0 \mid s_t, \theta)
\right]
\right\}
\end{gather*}
$$


### Nested fixed point algorithm of Expected Value Function
Because $EV_\theta (s, \varepsilon, a) = EV_\theta(s,a)$, we can rewrite the expected value function:


$$
\begin{gather*}
EV_\theta(s_t, a_t) = 
E\left[
\max_{a_{t+1}} \left\{
\pi^{m}(s_{t+1}, a_{t+1}) + \varepsilon_{t+1}(a_{t+1}) + \delta EV_{\theta}
(s_{t+1}, a_{t+1})
\right\} \mid s_t, a_t
\right]\\
= E_{s_{t+1}, \varepsilon_{t+1}\mid s_t, a_t} \left[
\max_{a_{t+1}}\left\{
\pi^{m}(s_{t+1}, a_{t+1}) + \varepsilon_{t+1}(a_{t+1}) + \delta EV_{\theta}
(s_{t+1}, a_{t+1})
\right\}
\right]\\
= E_{s_{t+1}\mid s_t, a_t}E_{\varepsilon_{t+1} \mid s_{t+1}, s_{t}, a_{t}}
\left[\cdots\right]\\
= E_{s_{t+1}\mid s_t, a_t} \log\left[
\sum_{a_{t+1}} \exp \left\{
\pi^{m}(s_{t+1}, a_{t+1}) + \delta EV_{\theta}(s_{t+1}, a_{t+1})
\right\}
\right]
\end{gather*}
$$

Since $s_{t+1} \sim P_{s_t, a_t}$, we obtain the following expression:
if $(s_t, a_t) = (3, 1)$,
$$
EV_{\theta}(3,1) =  \gamma \log\left[ \sum_{a_{t+1}} \exp \left\{\pi^{m}(4, a_{t+1}) + \delta EV_{\theta}(4, a_{t+1})
\right\}
\right] \\
+ (1 - \kappa - \gamma) \log\left[ \sum_{a_{t+1}} \exp \left\{\pi^{m}(3, a_{t+1}) + \delta EV_{\theta}(3, a_{t+1})
\right\}
\right] \\
+ \kappa \log\left[ \sum_{a_{t+1}} \exp \left\{\pi^{m}(2, a_{t+1}) + \delta EV_{\theta}(2, a_{t+1})
\right\}
\right]
$$

In [0]:
kappa = result.x[0]
gamma = result.x[1]
delta = 0.95
theta3 = [kappa, gamma]

# Transition prob. matrices
P0 = np.eye(5, dtype = float) - theta3[0]*np.eye(5, dtype = float)
P0[0,0] = 1
kap = np.eye(4, dtype = float)*theta3[0]
kap = np.insert(np.insert(kap, obj = 0, values = 0, axis = 0), obj = 4, values = 0, axis = 1) # 4 * 4 -> 5 * 4 -> 5 * 5
P0 += kap # defined transition prob. matrix when a = 0

gam = np.eye(4, dtype = float)*theta3[1]
gam_center = np.insert(np.insert(gam, obj = 4, values = 0, axis = 0), obj = 4, values = 0, axis = 1)
gam_up = np.insert(np.insert(gam, obj = 0, values = 0, axis = 1), obj = 4, values = 0, axis = 0)
P1 = P0 - gam_center + gam_up # defined transition prob. matrix when a = 1

In [0]:
### EV[0:4] <- EV(hoge, 0), EV[5:9] <- EV(hoge, 1)
EV = np.zeros(10) # container of values of expected value function.
init_theta1 = [2,2]

In [0]:
def new_value_func(EV, theta1):
  alpha = theta1[0]
  beta = theta1[1]
  num_s = len(EV)
  mean_pi_0 = np.array(range(num_s // 2))
  mean_pi_0 = np.log(mean_pi_0 + 1) * alpha
  mean_pi_1 = mean_pi_0 - beta
  EV_0 = EV[0:num_s//2]
  EV_1 = EV[num_s//2:num_s]
  
  ### calculate RHS.
  inside_bracket =np.exp(mean_pi_0 + delta* EV_0) + np.exp(mean_pi_1 + delta*EV_1)
  V = np_log(inside_bracket)
  EV_new_0 = np.matmul(P0, V)
  EV_new_1 = np.matmul(P1, V)
  diff = np.append(EV_new_0 - EV_0, EV_new_1 - EV_1)
  supdiff = np.max(np.abs(diff))
  return np.concatenate([EV_new_0, EV_new_1], axis = 0), supdiff

def inner_loop(init, theta1, x_tol = 1e-8, max_iter = 5000):
  EV_old = init
  success = False
  iteration = 0
  while (~(success) & (iteration < max_iter)):
    EV_new, diff = new_value_func(EV_old, theta1)
    success = (diff < x_tol)
    iteration += 1
    EV_old = EV_new
  if ~(success):
    print('Inner loop did not converge in {} iterations. Final difference is {:.3e}.'.format(iteration, diff))
  return EV_new

In [0]:
new_value_func(EV, init_theta1)

(array([0.12692801, 1.38701624, 2.25032673, 2.84713646, 3.30517454,
        0.86172629, 1.81684568, 2.55529557, 3.08368867, 3.30517454]),
 3.305174535833593)

In [0]:
EV_new = inner_loop(EV, init_theta1) 

Inner loop converged!  Num Iteration:382  difference:9.5406e-09


In [0]:
def get_logL(EV, theta1):
  alpha = theta1[0]
  beta = theta1[1]
  num_s = len(EV)
  mean_pi_0 = np.array(range(num_s // 2))
  mean_pi_0 = np.log(mean_pi_0 + 1) * alpha
  mean_pi_1 = mean_pi_0 - beta
  EV_0 = EV[0:num_s//2]
  EV_1 = EV[num_s//2:num_s]
  numerator_0 = mean_pi_0 + delta*EV_0
  numerator_1 = mean_pi_1 + delta*EV_1
  denom =np_log(np.exp(numerator_0) + np.exp(numerator_1))
  
  logL = np.append(numerator_0 - denom, numerator_1 - denom, axis = 0)
  return logL

def calc_object(EV, theta1):
  logL = get_logL(EV, theta1)
  data_trans['object'] = logL[data_trans['state']-1 + data_trans['action']*5]
  Object = data_trans['object'].sum(axis = 0)
  return -Object

In [0]:
nlog_likelihood = calc_object(EV_new, init_theta1)

In [0]:
print(nlog_likelihood)

90604.40155885558


In [0]:
def valuated_func(theta1):
  EV_init = np.zeros(10)
  EV_new = inner_loop(EV_init, theta1)
  nll = calc_object(EV_new, theta1)
  return nll

def outer_loop(max_iter = 5000):
  init_theta1 = [2,2]
  lin_const_out = LinearConstraint([[1, 0], [0, 1]], [0, 0], [np.inf, np.inf])
  result = minimize(valuated_func, init_theta1, method = 'trust-constr', constraints = lin_const_out) # これはなぜ要素が２つなのか...?
  return result
  

In [0]:
outer_result = outer_loop()
print(outer_result)

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 169
      cg_stop_cond: 2
            constr: [array([0.5501375 , 3.01088499])]
       constr_nfev: [0]
       constr_nhev: [0]
       constr_njev: [0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 4.815604209899902
               fun: 39963.836917078384
              grad: array([0.00195312, 0.00129738])
               jac: [array([[1, 0],
       [0, 1]])]
   lagrangian_grad: array([0.00045378, 0.00116848])
           message: '`xtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 225
              nhev: 0
               nit: 110
             niter: 110
              njev: 0
        optimality: 0.0011684811404220757
            status: 2
           success: True
         tr_radius: 9.174819772526787e-09
                 v: [array([-0.00149935, -0.00012889])]
                 x: array([0.5501375 , 3.01088499])


In [0]:
class Rust:
  def __init__(self, delta = 0.95):
    self.kappa = 0.2
    self.gamma = 0.1
    self.delta = delta
    self.theta1 = [2,2]
    self.theta3 = [self.kappa, self.gamma]
    self.EV_init = np.zeros(10)
    self.EV = np.zeros(10) # Initial value
    
  def NLL(self, theta3):
    self.theta3 = theta3
    # Transition prob. matrices
    self.P0 = np.eye(5, dtype = float) - self.theta3[0]*np.eye(5, dtype = float)
    self.P0[0,0] = 1
    kap = np.eye(4, dtype = float)*self.theta3[0]
    kap = np.insert(np.insert(kap, obj = 0, values = 0, axis = 0), obj = 4, values = 0, axis = 1) # 4 * 4 -> 5 * 4 -> 5 * 5
    self.P0 += kap # defined transition prob. matrix when a = 0
    # define P1
    gam = np.eye(4, dtype = float)*self.theta3[1]
    gam_center = np.insert(np.insert(gam, obj = 4, values = 0, axis = 0), obj = 4, values = 0, axis = 1)
    gam_up = np.insert(np.insert(gam, obj = 0, values = 0, axis = 1), obj = 4, values = 0, axis = 0)
    self.P1 = self.P0 - gam_center + gam_up # defined transition prob. matrix when a = 1
    # calculate negative log-likelihood
    dat['next_state'] = dat['state'].shift(periods = -1, axis = 0)
    data_trans = dat[dat['time'] != 100]
    data_trans = data_trans.astype('int8')
    data_trans['logL'] = np.where(data_trans['action'] == 1, np_log(self.P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), np_log(self.P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
    data_trans['L'] = np.where(data_trans['action'] == 1, (self.P1[data_trans['state'].astype('int8') -1, data_trans['next_state'].astype('int8')-1]), (self.P0[data_trans['state'].astype('int8')-1, data_trans['next_state'].astype('int8')-1]))
    logL = data_trans['logL'].sum()
    return -logL
  
  def fit_NLL(self):
    lin_const = LinearConstraint([[1, 1], [-1, 0], [0, -1]], [0, -1, -1], [1, 0, 0])
    result = minimize(self.NLL, [0.1, 0.2], method = 'trust-constr', constraints = lin_const) # これはなぜ要素が２つなのか...?
    return result
  
  def new_value_func(self, EV, theta1):
    alpha = theta1[0]
    beta = theta1[1]
    num_s = len(EV)
    mean_pi_0 = np.array(range(num_s // 2))
    mean_pi_0 = np.log(mean_pi_0 + 1) * alpha
    mean_pi_1 = mean_pi_0 - beta
    EV_0 = EV[0:num_s//2]
    EV_1 = EV[num_s//2:num_s]

    ### calculate RHS.
    inside_bracket =np.exp(mean_pi_0 + delta* EV_0) + np.exp(mean_pi_1 + delta*EV_1)
    V = np_log(inside_bracket)
    EV_new_0 = np.matmul(P0, V)
    EV_new_1 = np.matmul(P1, V)
    diff = np.append(EV_new_0 - EV_0, EV_new_1 - EV_1)
    supdiff = np.max(np.abs(diff))
    return np.concatenate([EV_new_0, EV_new_1], axis = 0), supdiff

  def inner_loop(self, init, theta1, x_tol = 1e-8, max_iter = 5000):
    EV_old = init
    success = False
    iteration = 0
    while (~(success) & (iteration < max_iter)):
      EV_new, diff = self.new_value_func(EV_old, theta1)
      success = (diff < x_tol)
      iteration += 1
      EV_old = EV_new
    if ~(success):
      print('Inner loop did not converge in {} iterations. Final difference is {:.3e}.'.format(iteration, diff))
    return EV_new
  
  def get_logL(self, EV, theta1):
    alpha = theta1[0]
    beta = theta1[1]
    num_s = len(EV)
    mean_pi_0 = np.array(range(num_s // 2))
    mean_pi_0 = np.log(mean_pi_0 + 1) * alpha
    mean_pi_1 = mean_pi_0 - beta
    EV_0 = EV[0:num_s//2]
    EV_1 = EV[num_s//2:num_s]
    numerator_0 = mean_pi_0 + delta*EV_0
    numerator_1 = mean_pi_1 + delta*EV_1
    denom =np_log(np.exp(numerator_0) + np.exp(numerator_1))
    logL = np.append(numerator_0 - denom, numerator_1 - denom, axis = 0)
    return logL

  def calc_object(self, EV, theta1):
    logL = self.get_logL(EV, theta1)
    data_trans['object'] = logL[data_trans['state']-1 + data_trans['action']*5]
    Object = data_trans['object'].sum(axis = 0)
    return -Object
  
  def valuated_func(self, theta1):
    self.EV = self.inner_loop(self.EV_init, theta1)
    nll = self.calc_object(self.EV, theta1)
    return nll

  def outer_loop(self, max_iter = 5000):
    lin_const_out = LinearConstraint([[1, 0], [0, 1]], [0, 0], [np.inf, np.inf])
    result = minimize(self.valuated_func, self.theta1, method = 'trust-constr', constraints = lin_const_out) # これはなぜ要素が２つなのか...?
    return result